In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from block2 import SZ as block2_SZ
from pathlib import Path
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2 import tools
from pyblock2._pyscf.ao2mo import integrals as itg
import pyscf
import openfermion as of
from pyscf import gto, scf
from openfermion import FermionOperator
from utils.physicist_to_chemist import physicist_to_chemist, chemist_ferm_to_tensor, physicist_to_chemist_tensor
from SolvableQubitHamiltonians.fermi_util import get_commutator

In [2]:
def construct_ob_spatial_fermion_operator(ferm_op, obt):

    num_orbitals = obt.shape[0]  # assuming h1e is square
    num_spin_orbs = 2 * num_orbitals
    for p in range(num_orbitals):
        for q in range(num_orbitals):
            coefficient = obt[p, q]
            if coefficient != 0.0:
                term = FermionOperator(((p, 1), (q, 0)), coefficient)
                ferm_op += term
    
    return ferm_op


def construct_tb_spatial_fermion_operator(ferm_op, tbt):
    num_orbitals = tbt.shape[0]  # assuming h1e is square
    for p in range(num_orbitals):
        for q in range(num_orbitals):
            for r in range(num_orbitals):
                for s in range(num_orbitals):
                    coefficient = tbt[p, q, r, s]
                    if coefficient != 0.0:
                        term_1 = FermionOperator(((p, 1), (q, 0), (r, 1), (s, 0)), coefficient)
                        
                        ferm_op += term_1
    
    return ferm_op

In [3]:
mol = gto.M(
    atom=f"H 0 0 0; H 0 0 1.0",
    basis='sto-3g')
mf = scf.RHF(mol).run(conv_tol=1E-14)
mycc = mf.CISD().run()
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
                                                                     ncore=0,
                                                                     g2e_symm=1)

converged SCF energy = -1.06610864931794
E(RCISD) = -1.101150330232619  E_corr = -0.03504168091468246


In [4]:
obt, tbt = physicist_to_chemist_tensor(h1e, g2e)

In [5]:
H = FermionOperator()
H = construct_ob_spatial_fermion_operator(H, obt)
H = construct_tb_spatial_fermion_operator(H, tbt)
print(H)

-1.5224407213912212 [0^ 0] +
0.3132012497647589 [0^ 0 0^ 0] +
0.31085338155985665 [0^ 0 1^ 1] +
0.0983952917427351 [0^ 1 0^ 1] +
0.0983952917427351 [0^ 1 1^ 0] +
0.0983952917427351 [1^ 0 0^ 1] +
0.0983952917427351 [1^ 0 1^ 0] +
-1.014051668920105 [1^ 1] +
0.31085338155985665 [1^ 1 0^ 0] +
0.3265353734712869 [1^ 1 1^ 1]


In [8]:
G = FermionOperator('1^ 0') - FermionOperator('0^ 1')
commutator = get_commutator(H, G)
print(commutator)

0.5217231761776443 [0^ 1] +
0.5217231761776444 [1^ 0]
